In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random

from scipy.io import loadmat
from matplotlib.ticker import MaxNLocator

In [ ]:
# Define fonts and fontsize for plotting
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
fontsize = 12

In [ ]:
# Define necessary functions
# Function to get displacement and void data
def get_data(file_directory, void_number):

    # Load the displacement data
    test_displacement_data = loadmat(os.path.join(file_directory, f'void_{void_number}_test_displacement_data_normalized.mat'))['displacement_data']
   
    # Load the void data
    test_void_data = loadmat(os.path.join(file_directory, f'void_{void_number}_test_void_data.mat'))['void_data']
    
    return test_displacement_data, test_void_data

In [ ]:
# Function to calculate metrics
def metrics_calculator(target, predicted):

    # True positive
    tp = 0

    # True negative
    tn = 0

    # False positive
    fp = 0

    # False negative
    fn = 0

    # Accuracy, Precision, Recall, F1 score
    accuracy = []
    precision = []
    recall = []

    for n_sample in range(target.shape[0]):

        for n_element in range(target.shape[1]):
            
            # If both the target and predicted are same
            if target[n_sample, n_element] == predicted[n_sample, n_element]:

                # If the target and predicted are both 1
                if target[n_sample, n_element] == 1 and predicted[n_sample, n_element] == 1:

                    tp = tp + 1

                # If the target and predicted are both 0
                elif target[n_sample, n_element] == 0 and predicted[n_sample, n_element] == 0:

                    tn = tn + 1
            
            # If the target and predicted are different
            else:

                # If the target is 0 and predicted is 1
                if target[n_sample, n_element] == 0 and predicted[n_sample, n_element] == 1:

                    fp = fp + 1
                
                # If the target is 1 and predicted is 0
                elif target[n_sample, n_element] == 1 and predicted[n_sample, n_element] == 0:
                
                    fn = fn + 1
            
        # Calculate accuracy, precision, and recall
        accuracy.append((tp + tn) / (tp + tn + fp + fn))
        precision.append(tp / (tp + fp))
        recall.append(tp / (tp + fn))
    
    # Convert the lists to numpy arrays
    accuracy = np.array(accuracy)
    precision = np.array(precision)
    recall = np.array(recall)

    # Calculate the F1 score
    f1_score = 2 * (precision * recall) / (precision + recall)

    return accuracy, precision, recall, f1_score

In [ ]:
# Function to plot void 0 prediction
def plot_void(target, predicted, file_directory, void):

    sample_num = random.randint(0, target.shape[0])

    # See how the actual void looks in target
    actual_plot = plt.figure()
    ax = plt.axes()
    im1 = plt.imshow(target[sample_num, :, :], cmap='binary', origin='lower', vmin=0, vmax=1, extent=[0, 50, 0, 50])
    
    # Grid lines
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.grid(which='both')
    
    plt.xlabel('x [m]', fontsize=fontsize)
    plt.ylabel('y [m]', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    cax = actual_plot.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
    cb = plt.colorbar(im1, cax=cax)
    cb.ax.tick_params(labelsize=fontsize)
    plt.savefig(os.path.join(file_directory, f'void_{void}_target.pdf'), bbox_inches='tight')

    # See how the predicted void looks in prediction
    predicted_plot = plt.figure()
    ax = plt.axes()
    im2 = plt.imshow(predicted[sample_num, :], cmap='binary', origin='lower', vmin=0, vmax=1, extent=[0, 50, 0, 50])
    
    # Grid lines
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.grid(which='both')

    plt.xlabel('x [m]', fontsize=fontsize)
    plt.ylabel('y [m]', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    cax = predicted_plot.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
    cb = plt.colorbar(im2, cax=cax)
    cb.ax.tick_params(labelsize=fontsize)
    plt.savefig(os.path.join(file_directory, f'void_{void}_predicted.pdf'), bbox_inches='tight')

    return actual_plot, predicted_plot

In [ ]:
# Define directories
current_directory = os.getcwd()

# Define directory for the normalized data
normalized_data_directory = os.path.join(current_directory, '..', '..', 'data', 'normalized')

# Define directory for the trained results
trained_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'training_results')

# Define directory for the prediction results
prediction_results_directory = os.path.join(current_directory, '..', '..', 'cnn', 'prediction_results', 'test')

In [ ]:
# Get the normalized data for all three voids
void = 0
void_test_displacement_data, void_target_void_data = get_data(normalized_data_directory, void)

In [ ]:
# Load the trained model
model = tf.keras.models.load_model(os.path.join(trained_results_directory, 'best_model.h5'))

In [ ]:
# Use the model to predict on the test data both probability and class
threshold = 0.5
void_predicted_void_data_probabilistic = model.predict(void_test_displacement_data)
void_predicted_void_data_class = np.where(void_predicted_void_data_probabilistic > threshold, 1, 0)

In [ ]:
# Plot the void 0 prediction
if void != 0:

    # Calculate the metrics
    accuracy, precision, recall, f1_score = metrics_calculator(void_target_void_data, void_predicted_void_data_class)

# Reshape the target and predicted data to a 50x50 grid
void_target_void_data = np.reshape(void_target_void_data, (void_target_void_data.shape[0], 50, 50))
void_predicted_void_data_probabilistic = np.reshape(void_predicted_void_data_probabilistic, (void_predicted_void_data_probabilistic.shape[0], 50, 50))

actual_plot, predicted_plot = plot_void(void_target_void_data, void_predicted_void_data_probabilistic, prediction_results_directory, void)